In [ ]:
import os
import zipfile
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# --- CONFIGURAZIONE AVANZATA ---
ZIP_NAME = "dataset.zip"
EXTRACTED_DIR = "dataset"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32 # Se la memoria GPU si riempie, scendi a 16

print("🛠️ --- FASE 1: PREPARAZIONE AMBIENTE ---")

# 1. Pulizia e Estrazione (Codice Robusto)
if os.path.exists(EXTRACTED_DIR):
    shutil.rmtree(EXTRACTED_DIR)

if os.path.exists(ZIP_NAME):
    print("📂 Estrazione dataset in corso...")
    with zipfile.ZipFile(ZIP_NAME, 'r') as zip_ref:
        zip_ref.extractall(".")
else:
    raise FileNotFoundError("❌ ERRORE: Carica 'dataset.zip' nella barra a sinistra!")

# Identificazione cartella (nel caso lo zip abbia un nome interno diverso)
if not os.path.exists(EXTRACTED_DIR):
    folders = [d for d in os.listdir() if os.path.isdir(d) and d not in ['.config', 'sample_data']]
    if len(folders) == 1:
        EXTRACTED_DIR = folders[0]
print(f"✅ Cartella dati: {EXTRACTED_DIR}")

# --- FASE 2: DATA AUGMENTATION ---
# Fondamentale per addestramenti lunghi: crea varianti per non far annoiare il modello
print("🔄 Configurazione Generatori Immagini...")

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    rotation_range=30,      # Aumentato a 30 gradi
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2    # 20% validation
)

train_generator = train_datagen.flow_from_directory(
    EXTRACTED_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    EXTRACTED_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# --- FASE 3: STRATEGIA DEI CALLBACKS (Il segreto del successo) ---
# Questi sono i "vigili" che controllano l'addestramento
callbacks_list = [
    # 1. Salva SOLO se batte il record precedente di precisione
    ModelCheckpoint(
        filepath="best_model.h5",
        monitor="val_accuracy",
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    # 2. Se non migliora per 12 epoche, fermati (evita spreco di tempo e overfitting)
    EarlyStopping(
        monitor="val_accuracy",
        patience=12,
        restore_best_weights=True,
        verbose=1
    ),
    # 3. Se ti blocchi (plateau) per 5 epoche, rallenta l'apprendimento per essere più preciso
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

# --- FASE 4: COSTRUZIONE MODELLO ---
print("\n🧠 Costruzione MobileNetV2...")

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x) # Aumentato Dropout al 40% per addestramenti lunghi
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# --- FASE 5: TRAINING PARTE 1 (WARM UP) ---
# Addestriamo solo la "testa" (i layer finali) tenendo MobileNet congelato
print("\n🚀 --- ROUND 1: Warm Up (Base congelata) ---")
base_model.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Facciamo 20 epoche di riscaldamento (o meno se EarlyStopping interviene)
history_warmup = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=callbacks_list # Usiamo già i controlli
)

# --- FASE 6: TRAINING PARTE 2 (FINE TUNING PROFONDO) ---
# Scongeliamo MobileNet per adattarlo specificamente ai rifiuti
print("\n🚀 --- ROUND 2: Fine Tuning Profondo (Base sbloccata) ---")

base_model.trainable = True

# Importante: Non scongelare tutto tutto, altrimenti distruggi le conoscenze base.
# Scongeliamo dagli ultimi layer fino al layer 50 (MobileNet ne ha circa 150)
# Più basso è il numero, più "libertà" dai al modello, ma più tempo ci mette.
fine_tune_at = 50

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Usiamo un learning rate BASSISSIMO per non fare danni
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Qui diamo "tempo infinito" (100 epoche), tanto ci pensano i Callbacks a fermarlo
history_fine = model.fit(
    train_generator,
    epochs=100,
    initial_epoch=history_warmup.epoch[-1], # Riparte da dove era arrivato
    validation_data=validation_generator,
    callbacks=callbacks_list
)

print("\n✅ ADDESTRAMENTO COMPLETATO!")
print("🏆 Il file 'best_model.h5' contiene la versione migliore ottenuta durante tutto il processo.")
print("Scaricalo dalla barra a sinistra (premi Aggiorna se non lo vedi).")

🛠️ --- FASE 1: PREPARAZIONE AMBIENTE ---
📂 Estrazione dataset in corso...
✅ Cartella dati: dataset
🔄 Configurazione Generatori Immagini...
Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.

🧠 Costruzione MobileNetV2...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

🚀 --- ROUND 1: Warm Up (Base congelata) ---


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 515ms/step - accuracy: 0.3898 - loss: 1.6252
Epoch 1: val_accuracy improved from -inf to 0.63618, saving model to best_model.h5


64/64 ━━━━━━━━━━━━━━━━━━━━ 73s 859ms/step - accuracy: 0.3917 - loss: 1.6201 - val_accuracy: 0.6362 - val_loss: 0.9336 - learning_rate: 0.0010
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.6960 - loss: 0.8112
Epoch 2: val_accuracy improved from 0.63618 to 0.70577, saving model to best_model.h5


64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 493ms/step - accuracy: 0.6960 - loss: 0.8111 - val_accuracy: 0.7058 - val_loss: 0.8182 - learning_rate: 0.0010
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step - accuracy: 0.7444 - loss: 0.6772
Epoch 3: val_accuracy improved from 0.70577 to 0.73757, saving model to best_model.h5


...

64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 467ms/step - accuracy: 0.8158 - loss: 0.5164 - val_accuracy: 0.7555 - val_loss: 0.6869 - learning_rate: 0.0010
Epoch 8/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.8196 - loss: 0.4799
Epoch 8: val_accuracy did not improve from 0.75547
64/64 ━━━━━━━━━━━━━━━━━━━━ 34s 536ms/step - accuracy: 0.8196 - loss: 0.4800 - val_accuracy: 0.7336 - val_loss: 0.6657 - learning_rate: 0.0010
Epoch 9/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.8241 - loss: 0.4906
Epoch 9: val_accuracy did not improve from 0.75547
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 458ms/step - accuracy: 0.8242 - loss: 0.4903 - val_accuracy: 0.7515 - val_loss: 0.6580 - learning_rate: 0.0010
Epoch 10/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.8117 - loss: 0.4958
Epoch 10: val_accuracy did not improve from 0.75547
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 473ms/step - accuracy: 0.8118 - loss: 0.4953 - val_accuracy: 0.7515 - val_loss: 0.6885 - learning_rate: 0.0010
Epoch 11/20
64/64 ━

64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 465ms/step - accuracy: 0.8587 - loss: 0.3889 - val_accuracy: 0.7654 - val_loss: 0.6593 - learning_rate: 0.0010
Epoch 16/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.8309 - loss: 0.4302
Epoch 16: val_accuracy did not improve from 0.76541
64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 480ms/step - accuracy: 0.8312 - loss: 0.4297 - val_accuracy: 0.7376 - val_loss: 0.7103 - learning_rate: 0.0010
Epoch 17/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.8736 - loss: 0.3657
Epoch 17: val_accuracy did not improve from 0.76541

Epoch 17: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 460ms/step - accuracy: 0.8735 - loss: 0.3660 - val_accuracy: 0.7575 - val_loss: 0.6528 - learning_rate: 0.0010
Epoch 18/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.8654 - loss: 0.3652
Epoch 18: val_accuracy improved from 0.76541 to 0.77932, saving model to best_model.h5


64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 485ms/step - accuracy: 0.8653 - loss: 0.3654 - val_accuracy: 0.7793 - val_loss: 0.6008 - learning_rate: 2.0000e-04
Epoch 19/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.8599 - loss: 0.3795
Epoch 19: val_accuracy did not improve from 0.77932
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 461ms/step - accuracy: 0.8598 - loss: 0.3797 - val_accuracy: 0.7694 - val_loss: 0.6726 - learning_rate: 2.0000e-04
Epoch 20/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.8510 - loss: 0.3803
Epoch 20: val_accuracy did not improve from 0.77932
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 475ms/step - accuracy: 0.8511 - loss: 0.3800 - val_accuracy: 0.7654 - val_loss: 0.6480 - learning_rate: 2.0000e-04
Restoring model weights from the end of the best epoch: 18.

🚀 --- ROUND 2: Fine Tuning Profondo (Base sbloccata) ---
Epoch 20/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 637ms/step - accuracy: 0.6431 - loss: 1.0897
Epoch 20: val_accuracy did not improve from 0.77932
64/64 ━━━━━━━━━━━━━━━━━━━━

64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 482ms/step - accuracy: 0.8252 - loss: 0.4663 - val_accuracy: 0.7833 - val_loss: 0.6374 - learning_rate: 1.0000e-06
Epoch 38/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step - accuracy: 0.8397 - loss: 0.4568
Epoch 38: val_accuracy did not improve from 0.78330
64/64 ━━━━━━━━━━━━━━━━━━━━ 32s 495ms/step - accuracy: 0.8397 - loss: 0.4568 - val_accuracy: 0.7614 - val_loss: 0.6367 - learning_rate: 1.0000e-06
Epoch 39/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step - accuracy: 0.8541 - loss: 0.4058
Epoch 39: val_accuracy did not improve from 0.78330
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 476ms/step - accuracy: 0.8541 - loss: 0.4059 - val_accuracy: 0.7654 - val_loss: 0.6414 - learning_rate: 1.0000e-06
Epoch 40/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - accuracy: 0.8550 - loss: 0.4077
Epoch 40: val_accuracy did not improve from 0.78330
64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 484ms/step - accuracy: 0.8549 - loss: 0.4079 - val_accuracy: 0.7734 - val_loss: 0.6244 - learning_rate: 1.0000e

64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 487ms/step - accuracy: 0.8492 - loss: 0.4166 - val_accuracy: 0.7873 - val_loss: 0.6489 - learning_rate: 1.0000e-06
Epoch 50/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step - accuracy: 0.8688 - loss: 0.3614
Epoch 50: val_accuracy did not improve from 0.78728
64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 491ms/step - accuracy: 0.8686 - loss: 0.3617 - val_accuracy: 0.7813 - val_loss: 0.6092 - learning_rate: 1.0000e-06
Epoch 51/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - accuracy: 0.8661 - loss: 0.3856
Epoch 51: val_accuracy did not improve from 0.78728
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 476ms/step - accuracy: 0.8660 - loss: 0.3857 - val_accuracy: 0.7694 - val_loss: 0.6683 - learning_rate: 1.0000e-06
Epoch 52/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step - accuracy: 0.8621 - loss: 0.3865
Epoch 52: val_accuracy did not improve from 0.78728
64/64 ━━━━━━━━━━━━━━━━━━━━ 31s 487ms/step - accuracy: 0.8621 - loss: 0.3863 - val_accuracy: 0.7694 - val_loss: 0.6307 - learning_rate: 1.0000e